# <center> Saturation vapour pressure functions

**Goal of the notebook:**
Investigate the different saturation vapour pressure functions available in Air Sea Flux code for the COARE 3.5 parameterization



**Motivation:**
The choice of the function has a significant effect for the latent heat flux and q10n for the S88 paramaterization (see table S15): is it the same for COARE 3.5?


***What is the saturation vapour pressure function?***

The saturation vapour pressure function $e_s$ is used to compute the humidity when the humidity input is given by a relative humidy percentage or a dewpoint temperature.

See equation S11 in the supplementary material.

***What is the default method?***

Buck method (2012)

***What are the different methods available?***

There are 13 formulations for $e_s$ in AirSeaFluxCode.

***What is reported in Biri et al. 2023 regarding the quantification of the effect of $e_s$ used?***

In Biri et al. (2023), they investigate: 
- the impact of another $e_s$ formulation (WMO (2018) in comparison to the default one (Buck (2012)) formulation - for the 10 different parameterisations (C35, S80, ...)
- the impact of the 12 alternative formulations in comparison to the default one (Buck (2012)) for the S88 formulation



# 1. Load the packages, AirSeaFluxCode and data 

In [1]:
import sys
import pandas as pd
import numpy as np

In [2]:
# path to AirSeaFluxCode algorithm
sys.path.append('../../ms_thesis/AirSeaFluxCode/AirSeaFluxCode-master/AirSeaFluxCode/src/')

In [3]:
from AirSeaFluxCode import AirSeaFluxCode

In [4]:
# dataset from TARSAN 2022 cruise - ship data

ds_tarsan_2022 = pd.read_csv('../Data/TARSAN_2022/jgofs_30minmean_2022.csv')


# 2. Compute the latent heat flux and humidity for the C35 parameterisation with different qmet 

Information about qmeth parameter:
>  qmeth : str
> >     is the saturation evaporation method to use amongst
            "HylandWexler","Hardy","Preining","Wexler","GoffGratch","WMO",
            "MagnusTetens","Buck","Buck2","WMO2018","Sonntag","Bolton",
            "IAPWS","MurphyKoop"]
             default is Buck2


In [5]:
hu = np.asarray(34.3408)
ht = np.asarray(19.2024)
hin = np.array([hu, ht, ht])

In [6]:
qmeth_array = ["HylandWexler","Hardy","Preining","Wexler","GoffGratch","WMO",
            "MagnusTetens","Buck","Buck2","WMO2018","Sonntag","Bolton",
            "IAPWS","MurphyKoop"]
             

In [7]:
# Loop to compute AirSeaFluxCode output for each qmeth
d = {qmeth: pd.DataFrame(AirSeaFluxCode(spd=np.asarray(ds_tarsan_2022['WindSpeed']),
               T=np.asarray(ds_tarsan_2022['AirTemp']),
               SST=np.asarray(ds_tarsan_2022['SST']),
               SST_fl='bulk',
               meth='C35',
               lat=np.asarray(ds_tarsan_2022['Lat']),
               hum=["rh",np.asarray(ds_tarsan_2022['RH'])],
               hin=hin,
               hout=10,
               Rl=np.asarray(ds_tarsan_2022['PIR']),
               Rs=np.asarray(ds_tarsan_2022['PSP']),
               cskin=1,
               skin="C35",
               wl=1,
               gust=[1,1.2,600,0.01], # x=1 follows Fairall et al. 2003 - ref paper for C35, beta,zi and ustb are the default parameters in ASFcode
               qmeth=qmeth,
               tol=['all', 0.01, 0.01, 1e-05, 1e-3, 0.1, 0.1], #default
               maxiter=10, #default
               out=0, #default
               L="tsrv", #default
               out_var=['q10n','qsea','qair','latent']
              )) 
     for qmeth in qmeth_array}

# 3. Check the flag

In [19]:
for qmeth, df in d.items():
    print(np.shape(np.where(df['flag']!='n')))

(1, 65)
(1, 65)
(1, 65)
(1, 65)
(1, 65)
(1, 65)
(1, 65)
(1, 64)
(1, 64)
(1, 64)
(1, 65)
(1, 65)
(1, 65)
(1, 65)
